# Deep Learning avec PyTorch

## Prise en main de Pytorch

Supports adaptés de Sylvain Lamprier (sylvain.lamprier@univ-angers.fr) , Nicolas Baskiotis (nicolas.baskiotis@sorbonne-univeriste.fr) et Benjamin Piwowarski (benjamin.piwowarski@sorbonne-universite.fr) -- MLIA/ISIR, Sorbonne Université

In [ ]:
import torch
print("La version de torch est : ",torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())

import numpy as np
import sklearn


La version de torch est :  2.1.1+cu121
Le calcul GPU est disponible ?  False


### <span class="alert-success"> Syntaxe

Le principal objet manipulé sous Pytorch est **torch.Tensor** qui correspond à un tenseur mathématique (généralisation de la notion de matrice en $n$-dimensions), très proche dans l'utilisation de **numpy.array**.   Cet objet est optimisé pour les calculs sur GPU ce qui implique quelques contraintes plus importantes que sous **numpy**. En particulier :
* le type du tenseur manipulé est très important et les conversions ne sont pas automatique (**FloatTensor** de type **torch.float**, **DoubleTensor** de type **torch.double**,  **ByteTensor** de type **torch.byte**, **IntTensor** de type **torch.int**, **LongTensor** de type **torch.long**). Pour un tenseur **t** La conversion se fait très simplement en utilisant les fonctions : **t.double()**, **t.float()**, **t.long()** ...
* la plupart des opérations ont une version *inplace*, c'est-à-dire qui modifie le tenseur plutôt que de renvoyer un nouveau tenseur; elles sont suffixées par **_** (**add_** par exemple).

[Documentation officielle](https://pytorch.org/docs/stable/tensors.html) pour la liste exhaustive des opérations.


In [ ]:
# Création de tenseurs et caractéristiques
## Créer un tenseur (2,3) à partir d'une liste
t1=torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))

## Créer un tenseur  tenseur rempli de 1 de taille 2x3x4
t2 = torch.ones([2,3,4])

## tenseur de zéros de taille 2x3 de type float
t3 = torch.zeros([2,3,4],dtype=torch.float)

## tirage uniforme entier entre 10 et 15, 
## remarquez l'utilisation du _ dans random pour l'opération inplace
t4 = torch.empty([2, 3],dtype=torch.int32).random_(10,16)

## tirage suivant la loi normale
t5 = torch.empty([3, 4],dtype=torch.float32).normal_()

## equivalent à zeros(3,4).normal_
t6 = torch.randn([3, 4])

## Création d'un vecteur de 3 flottants selon la loi de normale
t7 = torch.randn([1, 3])

## concatenation de tenseurs sur la dimension 0
t8_1 = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
t8_2 = torch.tensor(np.array([[7, 8, 9], [10,11,12]]))
t8 = torch.cat((t8_1,t8_2),0)

## concatenation de tenseurs  sur la dimension 1
t9_1 = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
t9_2 = torch.tensor(np.array([[7, 8, 9], [10,11,12]]))
t9 = torch.cat((t9_1,t9_2),1)

## Taille des tenseurs/vecteurs
t10 = torch.zeros([2,3,4],dtype=torch.float)
taille = t10.shape

## Conversion de type
t11 = torch.ones([2,3,4],dtype=torch.float)
t11_int = t11.to(torch.int32)

# Opérations élémentaires sur les tenseurs
## produit scalaire (et contrairement à numpy, que produit scalaire)
t12_1 = torch.tensor([1, 2, 3])
t12_2 = torch.tensor([4, 5, 6])
t12 = torch.dot(t12_1, t12_2)

## produit matriciel : utilisation de @ ou de la fonction mm
t13_1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
t13_2 = torch.tensor([[7, 8], [9, 10], [11, 12]])
t13 = t13_1@t13_2

## transposé
t14_1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
t14 = t14_1.t()

## index du maximum selon une dimension
t15 = torch.tensor([[1, 2, 3], [4, 5, 6]])
max_values, max_indices = torch.max(t15, dim=1)

## somme selon une dimension/de tous les éléments
t16 = torch.tensor([[1, 2, 3], [4, 5, 6]])
somme = torch.sum(t16,dim=1)

## moyenne selon  une dimension/sur tous les éléments
t17 = torch.tensor([[1, 2, 3], [4, 5, 12]]).float().mean(dim=1)

## changer les dimensions du tenseur (la taille totale doit être inchangée)
t18 = torch.tensor([[1, 2, 3], [4, 5, 6]])
t_resized = t18.view(3, 2) 

## somme/produit/puissance termes a termes
t19_1 = torch.tensor([1, 2, 3])
t19_2 = torch.tensor([4, 5, 6])

sum_result = t19_1 + t19_2
product_result = t19_1 * t19_2
puiss_result = t19_1**2
#print("sum "+str(sum_result)+" poduct "+str(product_result)+" puiss "+str(puiss_result))

## Soit un tenseur a de (2,3,4). Le recopier dans une version (2,3,3,4) avec les tenseurs (3,4) 
## a[0] et a[1] recopiés chacun 3 fois (avec expand)
# Créer le tenseur initial a de forme (2, 3, 4)
a = torch.rand((2,3,4))
print("Tenseur initial (a) de forme (2, 3, 4):")
print(a)
# Créer les tenseurs (3, 4)
a=a.view(2,1,3,-1).expand(-1,3,-1,-1).contiguous().view(-1,3,4)
print("Tenseur final:")
print(a)

### <span class="alert-success"> Régression linéaire  </span>

On souhaite apprendre un modèle de régression linéaire $f$ du type:  $f(x,w,b)=x.w^t+b$  avec $x\in \mathbb{R}^{{d}}$ un vecteur d'observations pour lequel on souhaite prédire une sortie $\hat{y} \in \mathbb{R}$, $w\in\mathbb{R}^{1,d}$ et $b\in \mathbb{R}$ les paramètres du modèle. 

Pour cela on dispose d'un jeu de données étiquetées $\{(x,y)\}$, que l'on découpe en jeu d'entraînement (80%) et de validation (20%). Dans cet exercice, on utilisera le jeu de données très classique *Boston*, le prix des loyers à Boston en fonction de caractéristiques socio-économiques des quartiers.

On considèrera un coût moindres carrés pour apprendre le modèle sur le jeu d'entraînement (avec $N$ le nombre de données d'entraînement et $(x^i,y^i)$ le i-ème couple de cet ensemble): $$w^∗,b^∗=argmin_{w,b}\frac{1}{N} \sum_{i=1}^N \|f(x^i,w,b)-y^i\|^2$$


* Fonction **flineaire(x,w,b)** qui calcule $f(x,w,b)=x.w^t+b$  avec $x\in \mathbb{R}^{{n\times d}},~w\in\mathbb{R}^{1,d}, b\in \mathbb{R}$
* Fonction **loss(x,w,b,y)** qui retourne le coût moindre carré du modèle linéaire utilisant **flineaire(x,w,b)** pour un batch de données $x$ et leurs images associées $y$. 
* Calcul du gradient de l'erreur en fonction de chacun des paramètres $w_i$ et b par la fonction **getGradient(x,w,b)**
* Descente de gradient et apprentissage des paramètres optimaux pour la regression linéaire.
* Entraînement sur 80% des données et test sur 20%


In [3]:
def flineaire(x,w,b):
    return (x@w.T) + b

def getLoss(x,w,b,y):
    y = y.view(-1,1) # n ligne et 1 colonne
    return torch.pow(flineaire(x,w,b)-y,2).mean()/2.0 # pow -> puissance

def getGradient(x, w, b, y):
    # Calcul de la prédiction
    y = y.view(-1,1)
    yhat = flineaire(x,w,b)
    diff = yhat - y    
    '''
    #version boucle
    g= torch.zeros_like(w)
    for i in range(w.size(-1)):
        g[0,i] = (x[:,i].view(-1,1)*diff.mean())
    return g, diff.mean()
    '''
    return ((x.t()@diff)/x.shape[0]).t(),(diff.mean())



In [ ]:
## Chargement des données California et transformation en tensor.
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing() ## chargement des données
data_x = torch.tensor(housing['data'],dtype=torch.float)
data_y = torch.tensor(housing['target'],dtype=torch.float)

print("Nombre d'exemples : ",data_x.size(0), "Dimension : ",data_x.size(1))
print(data_x,data_y)

torch.random.manual_seed(1)

#initialisation aléatoire de w et b
w = torch.randn(1,data_x.size(1),requires_grad=True)
b =  torch.randn(1,1,requires_grad=True)

#decoupage en train 80 et test 20
train_x = data_x[:int(data_x.size(0)*0.8)]
train_y = data_y[:int(data_y.size(0)*0.8)]

test_x = data_x[int(data_x.size(0)*0.8):]
test_y = data_y[int(data_y.size(0)*0.8):]

print("Nombre d'exemples train : ",train_x.size(0), "Dimension : ",train_x.size(0),"x",train_x.size(1))
print("Nombre d'exemples test : ",test_x.size(0), "Dimension : ",test_x.size(0),"x",test_x.size(1))


EPOCHS = 100000
EPS = 1e-7

for i in range(EPOCHS):
    grad_w,grad_b = getGradient(train_x,w,b,train_y)

    w = w - EPS*grad_w
    b = b - EPS*grad_b

    loss = getLoss(train_x,w,b,train_y)
    
    if i%10000==0:
        print("Epoch : ",i," Loss : ",loss.item())
    